Audio classification - just like with text - assigns a class label output from the input data. The only difference is instead of text inputs, you have raw audio waveforms. Some practical applications of audio classification include identifying speaker intent, language classification, and even animal species by their sounds.

This guide shows how to:
1. Finetune Wav2Vec2 on the MInDS-14 dataset to classify speaker intent.
2. Use finetuned model for inference.

# Libraries

In [ ]:
pip install transformers datasets evaluate